In [1]:
#Load modules
import os
import pandas as pd
import numpy as np

In [11]:
#Set filenames
dataDir = '../../Data'
usageFN = dataDir + os.sep + "AllUsageData.csv"
supplyFN = dataDir + os.sep + "SupplyData.csv"
outFN = dataDir + os.sep + "UseAndSupply.csv"

In [3]:
#Create data frames from usage and supply
dfUsage = pd.read_csv(usageFN,dtype={'FIPS':np.str})
dfSupply = pd.read_csv(supplyFN,dtype={'FIPS':np.str})
print dfUsage.shape, dfSupply.shape

(9670, 12) (9156, 6)


In [4]:
dfSupply.head()

,YEAR,FIPS,total_runoff,pr,et,smc
0,2000,01001,107.792999,2803.677734,2632.648926,10212.400391
1,2000,01003,887.533020,10476.563477,9773.170898,60767.199219
2,2000,01005,459.471008,5946.130859,5752.579102,37530.699219
3,2000,01007,291.713989,3308.612061,3107.584961,30067.199219
4,2000,01009,49.201000,1805.101562,1781.492065,8463.200195


In [5]:
#Add State FIPS codes columns
dfUsage['STFIPS'] = dfUsage['FIPS'].apply(lambda x: str(x)[:2] )
dfSupply['STFIPS'] = dfSupply['FIPS'].apply(lambda x: str(x)[:2] )

In [6]:
#Compute supply as precip - et
dfSupply['supply_mm'] = dfSupply['pr'] - dfSupply['et']

In [7]:
#Drop the FIPS from the tables
dfUsage.drop('FIPS', axis=1, inplace = True)
dfSupply.drop(['FIPS','total_runoff','pr','et','smc'], axis=1, inplace = True)

In [8]:
#Group the supply table on state
grpSupply = dfSupply.groupby(['YEAR','STFIPS']).sum()

In [9]:
#Group the usage table on state
grpUsage = dfUsage.groupby(['YEAR','STFIPS']).sum()

In [12]:
#join tables on year and fips
dfX = pd.merge(grpSupply, grpUsage, how='outer',left_index=True,right_index=True)
dfX.to_csv(outFN,na_rep=0)

In [22]:
#Create FIPS lookup table
import urllib2
theURL = 'https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt'
data = urllib.urlopen(theURL).read()
from StringIO import StringIO
data2 = StringIO(data)
dfFIPS = pd.read_table(data)
dfFIPS.head()

HTTPError: HTTP Error 404: Not Found